In [1]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from datetime import datetime,timedelta
from out_in_bb import setup_1,setup_2
from backtesting import Strategy, Backtest
import warnings
warnings.filterwarnings("ignore")

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

#ativo = 'EURUSD'
ativo = 'WIN@N'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 60000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
df

,Data,Open,High,Low,Close,Vol
0,2022-08-08 13:25:00,108250.0,108285.0,108240.0,108280.0,18269
1,2022-08-08 13:26:00,108280.0,108420.0,108275.0,108395.0,74771
2,2022-08-08 13:27:00,108400.0,108410.0,108335.0,108350.0,27905
3,2022-08-08 13:28:00,108345.0,108350.0,108295.0,108295.0,26065
4,2022-08-08 13:29:00,108300.0,108370.0,108295.0,108335.0,22022
...,...,...,...,...,...,...
59995,2023-01-13 18:21:00,112055.0,112060.0,111985.0,112010.0,3772
59996,2023-01-13 18:22:00,112010.0,112080.0,111970.0,111980.0,5145
59997,2023-01-13 18:23:00,111985.0,112025.0,111960.0,112025.0,2927
59998,2023-01-13 18:24:00,112010.0,112030.0,111955.0,111995.0,2319


In [3]:
time_period = 40
var_bb = 2.3
pontos = 150
rate_stop = 1
rate_tp = 2
timeframe = 5

for time_period in [20,30,50]:
    for var_bb in [1.8,2,2.2]:
        for pontos in [100,150,200,250,300]:
            df_acao = setup_1(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_stop = rate_stop,rate_tp = rate_tp,timeframe = 5)
            #df_acao = setup_1_old(df,var_bb = 2,time_period = time_period,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)
            #df_acao = df[(df['acao'] == 'call')]

            df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
            df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])


            print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos} -- Timeframe: {timeframe}')
            df_new_hora = df_acao[['Hora_h','resultado_binario']]
            df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

            #df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
            print(df_new_hora)

            '''df_acao = setup_2(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_stop = rate_stop,rate_tp = rate_tp,timeframe = 5)
            #df_acao = df[(df['acao'] == 'call')]

            df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
            df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])


            print('Agrupado por Hora')
            df_new_hora = df_acao[['Hora_h','resultado_binario']]
            df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

            #df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
            print(df_new_hora)'''


Time_period: 20 -- var_bb: 1.8 -- pontos: 100 -- Timeframe: 5
  Hora_h  qtd_operacoes  taxa_acerto
0     09            368     0.334239
1     10            436     0.341743
2     11            350     0.371429
3     12            373     0.289544
4     13            332     0.358434
5     14            360     0.363889
6     15            362     0.356354
7     16            379     0.327177
8     17            353     0.320113
9     18             53     0.301887
Time_period: 20 -- var_bb: 1.8 -- pontos: 150 -- Timeframe: 5
  Hora_h  qtd_operacoes  taxa_acerto
0     09            368     0.304348
1     10            436     0.337156
2     11            350     0.360000
3     12            373     0.292225
4     13            332     0.349398
5     14            360     0.361111
6     15            362     0.323204
7     16            379     0.324538
8     17            353     0.348442
9     18             53     0.415094
Time_period: 20 -- var_bb: 1.8 -- pontos: 200 -- Timeframe: 5


In [9]:
df_acao[['Data','pos_BB_inicial','acao','resultado_binario']].tail(10)

,Data,pos_BB_inicial,acao,resultado_binario
39868,2023-01-11 17:20:00,0,sell,0.0
39874,2023-01-11 17:50:00,0,sell,0.0
39909,2023-01-12 11:20:00,0,call,1.0
39916,2023-01-12 11:55:00,0,call,0.0
39919,2023-01-12 12:10:00,0,call,1.0
39932,2023-01-12 13:15:00,0,sell,0.0
39959,2023-01-12 15:30:00,0,call,0.0
39973,2023-01-12 16:40:00,0,call,0.0
39980,2023-01-12 17:15:00,0,call,1.0
39996,2023-01-13 09:10:00,0,call,0.0
